In [3]:
import inspect
import time
import warnings

import matplotlib.pyplot as plt
import pandas as pd
#import plotly.express as px
import seaborn as sns
from IPython.display import VimeoVideo
#from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg

warnings.filterwarnings("ignore")

In [ ]:
client = MongoClient(host="localhost",port=27017)
db = client["air-quality"]
dar = db["dar-es-salaam"]

In [ ]:
sites = dar.distinct("metadata.site")
sites

In [ ]:
result = dar.find({"metadata.site":11,"metadat.measurement":"P2"},
                 projection={"P2":1,"timestamp":1,"_id":0}
                 )
readings_per_site =  list(dar.aggregate(
    [
        {"$group": {"_id": "$metadata.site", "count": {"$count": {}}}}
    ]
))
readings_per_site

In [ ]:
result = dar.find_one({})
result

In [ ]:
def wrangle(collection):
    result = collection.find({"metadata.site":11,"metadata.measurement":"P2"},
                 projection={"P2":1,"timestamp":1,"_id":0}
                 )
    
    #convert to dataframe
    df = pd.DataFrame(result).set_index("timestamp")
    
    #Localize reading time stamps to the timezone for "Africa/Dar_es_Salaam"
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")
    
    #Remove all outlier PM2.5 readings that are above 100
    df = df[df["P2"]<100]
    
    #Resample the data to provide the mean PM2.5 reading for each hour.
    y = df["P2"].resample("1H").mean().fillna(method="ffill")
    
    return y

In [ ]:
y.isna().sum()
y = wrangle(dar)
y.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.plot(ax=ax)
plt.xlabel("Date")
plt.ylabel("PM2.5 Level")
plt.title("Dar es Salaam PM2.5 Levels")
# Don't delete the code below 👇
plt.savefig("images/3-5-5.png", dpi=150)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.rolling(168).mean().plot(ax=ax)
plt.xlabel("Date")
plt.ylabel("PM2.5 Level")
plt.title("Dar es Salaam PM2.5 Levels, 7-Day Rolling Average")


In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y,ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam PM2.5 Readings")


In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y,ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam PM2.5 Readings PACF")



In [ ]:
cutoff_test = int(len(y)*0.9)
y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean]*len(y_train)
mae_baseline = mean_absolute_error(y_train,y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

In [ ]:
p_params = range(1, 31 )
maes = []
for p in p_params:
    model = AutoReg(y_train.copy(),lags=p).fit()
    y_pred = model.predict().dropna()
    mean_error = mean_absolute_error(y_train.loc[y_pred.index],y_pred)
    maes.append(mean_error)
mae_series = pd.Series(maes, name="mae", index=p_params)
mae_series.head()

In [ ]:
#AR autoregressive
p_params = range(1, 31)
maes = []
for p in p_params:
order = (1,0,p)
# Note start time
start_time = time.time()
# Train model
model = AutoReg(y_train, lags=p, old_names=True).fit()
# Calculate model training time
elapsed_time = round(time.time() - start_time, 2)
print(f"Trained AutoReg {order} in {elapsed_time} seconds.")
# Generate in-sample (training) predictions
y_pred = model.predict().dropna()
# Calculate training MAE
mae = mean_absolute_error(y_train.loc[y_pred.index], y_pred)
# Append MAE to list in dictionary
maes.append(mae)

mae_series = pd.Series(maes, name="mae", index=p_params)
mae_series.head()

In [ ]:
mea_df = pd.DataFrame(maes)
sns.heatmap(mae_df,cmap="Blues")

In [ ]:
best_p = 28
best_model = AutoReg(y_train, lags=best_p).fit()

In [ ]:
y_train_resid = model.resid
y_train_resid.name = "residuals"
y_train_resid.head()

In [ ]:
# Plot histogram of residuals
y_train_resid.hist()
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.title("Best Model, Training Residuals")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y_train_resid, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam, Training Residuals ACF")

In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    model = AutoReg(history,lags=28).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

In [ ]:
df_pred_test = pd.DataFrame({"y_test":y_test,"y_pred_wfv": y_pred_wfv})
fig = px.line(df_pred_test)
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)
# Don't delete the code below 👇
fig.write_image("images/3-5-18.png", scale=1, height=500, width=700)

fig.show()

In [ ]:
df_pred_test.head()